# Soenderborg Projekt 1 - English
This notebook is for assistance with the coding for many of the questions in the project.
The sections are marked with the corresponding question in the Project description.
Remember, this code is provided to get started with the project, but the code is not complete for answering the corresponding questions
#### Initialize python packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

#### Read data 

In [ ]:
# Path to project data (replace with your own path)
file_path = '/Users/johndoe/Documents/DTU/intro_stat/projects/soenderborg1/soenderborg1_data.csv'

## Read data into a pandas DataFrame
D = pd.read_csv(file_path, delimiter=";")

#### a) Simple summary of data

In [ ]:
print(f"Dimension of DataFrame: {D.shape}") # f-strings allow us to insert variables directly into the string
print(f"Variable names: {D.columns}")
print("\nFirst few rows of DataFrame:") # \n is the newline character for strings
display(D.head())
print("Last row of DataFrame:")
display(D.tail())
print("Some summary statistics:")
display(D.describe())
print("Data types:", D.dtypes)


#### b) Histogram (empirical)
Histogram describing the empirical density of the daily heat consumptions of House 1

In [ ]:
plt.hist(D['Q1'].dropna(), bins=20, density=True, color='blue', edgecolor='black') # dropna() removes potential missing values
plt.xlabel("Heat consumption (House 1)")
plt.ylabel("Density")
plt.show()

#### Date variable t

In [ ]:
D['t'] = pd.to_datetime(D['t'])
# to_datetime() method converts string to a datetime pandas object. 
# This is necesary to make it ordinal
display(D['t'].describe())

#### c) Plots of data over time

In [ ]:
houses = ['Q1', 'Q2', 'Q3', 'Q4']
colors = ['red', 'green', 'blue', 'purple']

# Plot of heat consumption over time
for i in range(4):
    plt.plot(D['t'], D[houses[i]], color=colors[i], label=houses[i])

plt.xlim(pd.to_datetime(["2008-10-02", "2010-10-01"]))
plt.ylim(0, 9)
plt.xlabel("Date")
plt.ylabel("Heat consumption")
plt.legend()
plt.show()


#### Data Subsets

In [ ]:
# Subset of the data: only Jan-Feb 2010
D_sub = D[(D['t'] >= "2010-01-01") & (D['t'] < "2010-03-01")]
display(D_sub.head())

#### d) Boxplot by House

In [ ]:
houses = ['Q1', 'Q2', 'Q3', 'Q4']
# Daily heat consumption by house
plt.boxplot([D_sub[house].dropna() for house in houses], labels=houses)
plt.xlabel("House")
plt.ylabel("Heat consumption")
plt.show()

#### e) Key summary statistics for House 1

In [ ]:
# Remember, we are still working with the subset D_sub (jan-feb 2010)
print(f"Total number of observations (without missing values): {D_sub['Q1'].notna().sum()}")
print(f"Sample mean of weekly returns: {np.mean(D_sub['Q1'])}")
print(f"Sample variance of weekly returns: {np.var(D_sub['Q1'], ddof=1)}") # ddof=1 as we want the *sample* variance

#### f) QQ-plot for model validation

In [ ]:
# QQ plot for House 1
sm.qqplot(D_sub['Q1'].dropna(), line='q')
plt.show()

#### g-h) One-sample t-test

In [ ]:
# Test hypothesis mu = 2.38 for daily heat consumption of House 1
res = stats.ttest_1samp(D_sub['Q1'].dropna(), popmean = 2.38)
print(f"t-statistic: {res.statistic}")
print(f"p-value: {res.pvalue}")

# Confidence interval
print(res.confidence_interval())

#### i) Welch t-test

#### Compare mean heat consumption of House 1 and House 2 with Welch t-test (for comparison of two independent means)

In [ ]:
# Compare daily heat consumption of House 1 and House 2
rest = stats.ttest_ind(D_sub['Q1'].dropna(), D_sub['Q2'].dropna(), equal_var=False)
print(f"t-statistic: {rest.statistic}")
print(f"p-value: {rest.pvalue}")

#### k) Correlation

In [ ]:
# Calculation of correlation between house 1 daily heat consumption and 
# global radiation
print(D[["Q1", "G"]].corr())

## EXTRA
#### Subsets in Python

In [ ]:
## Optional extra remark about taking subsets in R
##
## A logical vector with TRUE or FALSE for each row in D, e.g.:
## Finding days with frost
frost_days = D['Ta'] < 0
print("logical vector: \n", frost_days)
## Can be used to extract heat consumptions for House 1 on 
## days with frost
display(D['Q1'][frost_days])
## The .loc function can be used as well (all data on frost days)
print("Using .loc:")
display(D.loc[D['Ta'] < 0, :]) # ":" means all columns

## More complex logical expressions can be made, e.g.:
## Observations from days with frost before 2010
print("Frost days before 2010:")
display(D.loc[(D['Ta'] < 0) & (D['t'] < "2010-01-01"), 'Q1'])

# "display()" function gives a nicer table than print. It is 
# especially useful when working with dataframes (pandas)

#### Additional Python tips

In [ ]:
## Make a for loop to calculate some summary statistics for each house
Tbl = pd.DataFrame()
for i in ['Q1', 'Q2', 'Q3', 'Q4']:
    Tbl.loc[i, 'heat_mean'] = D[i].mean()
    Tbl.loc[i, 'heat_var'] = D[i].var(ddof=1)

display(Tbl)

In [ ]:
# There are many other ways to do these calculations, some more concise. For example
results = D.drop(columns=['t']).agg(['mean', 'var']) # drop the 't' column before calculating
# The agg function (aggregate) is used to calculate the mean and variance 
display(results)

# See more functions in pandas documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
# Numpy documentationen: https://numpy.org/doc/stable/reference/index.html
# Or find documentation or guides on other python packages/functions online.

#### Latex Tips
Pandas (pd) also includes a function that is very handy for writing tables/dataframes directly into Latex-code. 
This is done by usind the function `pd.to_latex()`.
The following is the simplest form of the function:

In [ ]:
Tbl_latex = Tbl.to_latex()
print(Tbl_latex)